In [1]:
using CSV, DataFrames, Distances, DelimitedFiles, Plots, JuMP, Cbc, Gurobi

In [2]:
function calc_score(cities, list_path, tenth)
    @views xy_cities = cities.xy
    len_path     = length(list_path)
    # Calc Distance
    @views xy_path   = xy_cities[list_path,:]
    @views @inbounds dist_path = sqrt.(sum((xy_path[1:end-1,:] .- xy_path[2:end,:]).^2; dims=2))[:,1]
    
    # List of Primes 0 to (len_path-1)
    # Flag array, is path's from-city number non-prime?
    @views is_path_from_non_prime   = cities.nprimes[list_path][1:end-1]   
    # If both flags are true, *1.1, else * 1.0
    result = dist_path .* (1.0 .+ 0.1 .* is_path_from_non_prime .* tenth)
    return Score(sum(result), dist_path, result)
end

calc_score (generic function with 1 method)

In [48]:
function solved(m,x,N)
    x_val = getvalue(x)
    
    # find cycle
    cycle_idx = Int[]
    push!(cycle_idx, 1)
    while true
        v, idx = findmax(x_val[cycle_idx[end],1:N])
        if idx == cycle_idx[1]
            break
        else
            push!(cycle_idx,idx)
        end
    end
#     println("cycle_idx: ", cycle_idx)
#     println("Length: ", length(cycle_idx))
    if length(cycle_idx) < N
        @constraint(m, sum(x[cycle_idx,cycle_idx]) <= length(cycle_idx)-1)
        return false
    end
    return true
end 

solved (generic function with 2 methods)

In [44]:
function run_mip(start, cities, primes, subm_path, N)
    if start % 2000 == 0
        println("Start: ", start)
    end
    
    pre_time = time()
    # generate distance matrix for the N cities
    c_pos = [zeros(2) for _ in 1:N]

    for i = 1:N
        c_pos[i] = [cities[:X][subm_path[start+i]+1],cities[:Y][subm_path[start+i]+1]]
    end
    dists = zeros(N,N)
    for i=1:N
        for j=i+1:N
            dists[i,j] = euclidean(c_pos[i],c_pos[j])
            dists[j,i] = dists[i,j]
        end
    end
    # it should be circular so N -> 1 has zero costs
    dists[N,1] = 0
  
    m = Model(solver=CbcSolver())
    @variable(m, x[1:N,1:N], Bin)
    @objective(m, Min, sum(x[i,j]*dists[i,j] for i=1:N,j=1:N))
    @constraint(m, notself[i=1:N], x[i,i] == 0)
    @constraint(m, oneout[i=1:N], sum(x[i,1:N]) == 1)
    @constraint(m, onein[j=1:N], sum(x[1:N,j]) == 1)
    for f=1:N, t=1:N
        @constraint(m, x[f,t]+x[t,f] <= 1)
    end
    # N has to be connected to 1
    @constraint(m, x[N,1] == 1)

    t = time()
    status = solve(m)

    while !solved(m,x,N)
        status = solve(m)
    end
    println("Time for solving: ", time()-t)

    println("Obj: ", getobjectivevalue(m))

    #=
    for f=1:N, t=1:N
        @constraint(m, x[f,t]+x[t,f] <= 1)
    end
    =#
#     println("Time before solve: ", time()-pre_time)
    
    status = solve(m)
    println("Status: ", status)
            
#     println("Post time: ", time()-post_time)
        
#     println("Obj: ", getobjectivevalue(m))
    # println("Cus to Fac: ",getvalue(cf))

    return [],false # subm_path, improved
end           

run_mip (generic function with 2 methods)

In [50]:
function main(from, to)
    cities = CSV.read("cities_p.csv");
    primes = findall(cities[:primes] .== true);
    subm_df = CSV.read("tsp_improved_mip.csv");

    subm_path = collect(skipmissing(subm_df[:Path]));
    for i=from:5:to
        subm_path, improved = run_mip(i, cities, primes, subm_path, 42);
        break
        if improved
           df = DataFrame(Path=subm_path)
           CSV.write("tsp_improved_mip.csv", df);
        end
    end
end

main (generic function with 1 method)

In [52]:
main(3124,100000)

Time for solving: 0.0426790714263916
Obj: 304.1062908181468
Status: Optimal
Presolve 0 (-1891) rows, 0 (-1764) columns and 0 (-7057) elements
Optimal - objective value 304.10629
After Postsolve, objective 304.10629, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 304.1062908 - 0 iterations time 0.002, Presolve 0.00
Cbc0045I Solution with objective value 304.106 saved
